In [80]:

#Upload shampoo-sales.csv
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving BitcoinDayPrice_April2013-June2018.npy to BitcoinDayPrice_April2013-June2018 (2).npy
User uploaded file "BitcoinDayPrice_April2013-June2018.npy" with length 30368 bytes


# LSTM Network and Data Preprocessing


LSTM for shampoo sales prediction tutorial: 
(https://)https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

Data: https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv


In [0]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

# date-time parsing function for loading the dataset
def parser(x):
  return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=10):
  df = DataFrame(data)
  columns = [df.shift(i) for i in range(1, lag+1)]
  columns.append(df)
  df = concat(columns, axis=1)
  df.fillna(0, inplace=True)
  return df

# create a differenced series
def difference(dataset, interval=1):
  diff = list()
  for i in range(interval, len(dataset)):
    value = dataset[i] - dataset[i - interval]
    diff.append(value)
  return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
  return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
  # fit scaler
  scaler = MinMaxScaler(feature_range=(-1, 1))
  scaler = scaler.fit(train)
  # transform train
  train = train.reshape(train.shape[0], train.shape[1])
  train_scaled = scaler.transform(train)
  # transform test
  test = test.reshape(test.shape[0], test.shape[1])
  test_scaled = scaler.transform(test)
  return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
  new_row = [x for x in X] + [value]
  array = numpy.array(new_row)
  array = array.reshape(1, len(array))
  inverted = scaler.inverse_transform(array)
  return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
  X, y = train[:, 0:-1], train[:, -1]
  X = X.reshape(X.shape[0], 1, X.shape[1])
  model = Sequential()
  model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  for i in range(nb_epoch):
    model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
    model.reset_states()
    print('Epoch ' + str(i) + ' complete')
  return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
  X = X.reshape(1, 1, len(X))
  yhat = model.predict(X, batch_size=batch_size)
  return yhat[0,0]

# load dataset
series = numpy.load('BitcoinDayPrice_April2013-June2018.npy')
raw_values = series[:,1]

# series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
            
# transform data to be stationary
# raw_values = series.values

diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
train_length = int((0.99*supervised_values.shape[0])) #Train on 99% of data
train, test = supervised_values[0:train_length], supervised_values[train_length:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

  

#Train Model

In [0]:
# fit the model
lstm_model = fit_lstm(train_scaled, 1, 500, 1000) #input: scaled training data, batch)size, # epochs, # neurons in hidden layer
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

print("Training complete.")


Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete
Epoch 10 complete
Epoch 11 complete
Epoch 12 complete
Epoch 13 complete
Epoch 14 complete
Epoch 15 complete
Epoch 16 complete
Epoch 17 complete
Epoch 18 complete
Epoch 19 complete
Epoch 20 complete
Epoch 21 complete
Epoch 22 complete
Epoch 23 complete
Epoch 24 complete
Epoch 25 complete
Epoch 26 complete
Epoch 27 complete
Epoch 28 complete
Epoch 29 complete
Epoch 30 complete
Epoch 31 complete
Epoch 32 complete
Epoch 33 complete
Epoch 34 complete
Epoch 35 complete
Epoch 36 complete
Epoch 37 complete
Epoch 38 complete
Epoch 39 complete
Epoch 40 complete
Epoch 41 complete
Epoch 42 complete
Epoch 43 complete
Epoch 44 complete
Epoch 45 complete
Epoch 46 complete
Epoch 47 complete
Epoch 48 complete
Epoch 49 complete
Epoch 50 complete
Epoch 51 complete
Epoch 52 complete
Epoch 53 complete
Epoch 54 complete
Epoch 55 complete
Ep

Epoch 105 complete
Epoch 106 complete
Epoch 107 complete
Epoch 108 complete
Epoch 109 complete
Epoch 110 complete
Epoch 111 complete
Epoch 112 complete
Epoch 113 complete
Epoch 114 complete
Epoch 115 complete
Epoch 116 complete
Epoch 117 complete
Epoch 118 complete
Epoch 119 complete
Epoch 120 complete
Epoch 121 complete
Epoch 122 complete
Epoch 123 complete
Epoch 124 complete
Epoch 125 complete
Epoch 126 complete
Epoch 127 complete
Epoch 128 complete
Epoch 129 complete
Epoch 130 complete
Epoch 131 complete
Epoch 132 complete
Epoch 133 complete
Epoch 134 complete
Epoch 135 complete
Epoch 136 complete
Epoch 137 complete
Epoch 138 complete
Epoch 139 complete
Epoch 140 complete
Epoch 141 complete
Epoch 142 complete
Epoch 143 complete
Epoch 144 complete
Epoch 145 complete
Epoch 146 complete
Epoch 147 complete
Epoch 148 complete
Epoch 149 complete
Epoch 150 complete
Epoch 151 complete
Epoch 152 complete
Epoch 153 complete
Epoch 154 complete
Epoch 155 complete
Epoch 156 complete
Epoch 157 co

#Predict and Report

In [0]:

# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
  # make one-step forecast
  X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
  yhat = forecast_lstm(lstm_model, 1, X)
  # invert scaling
  yhat = invert_scale(scaler, X, yhat)
  # invert differencing
  yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
  # store forecast
  predictions.append(yhat)
  expected = raw_values[len(train) + i + 1]
  print('Predicted = $%d, Expected = $%d' % (yhat, expected))
  
# report performance
rmse = sqrt(mean_squared_error(raw_values[-len(test_scaled):], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(raw_values[-len(test_scaled):], label = 'Expected')
pyplot.plot(predictions, label = 'Predicted')
pyplot.legend()
pyplot.show()